In [1]:
from flask import Flask, request, render_template, redirect, url_for, jsonify
from pymongo import MongoClient
from bson.objectid import ObjectId
import time
import datetime

app = Flask(__name__)
client = MongoClient(host="localhost", port=27017)
db = client.tic_tac_toe

In [2]:
board = ['' for _ in range(9)]
player_1 = ''
player_2 = ''
current_player = 'O'
winning_combinations = [{0, 1, 2}, {3, 4, 5}, {6, 7, 8}, {0, 3, 6}, {1, 4, 7}, {2, 5, 8}, {0, 4, 8}, {2, 4, 6}]
player_1_moves = set()
player_2_moves = set()
moves_sequence = []

In [3]:
def get_game_state():
    if any(combination.issubset(player_1_moves) for combination in winning_combinations):
        print('Player 1 wins')
        return 1
    if any(combination.issubset(player_2_moves) for combination in winning_combinations):
        print('Player 2 wins')
        return 2
    if len(player_1_moves) + len(player_2_moves) == 9:
        print('Draw')
        return 3
    return 0

def reset_game():
    global board, current_player, player_1_moves, player_2_moves, moves_sequence
    board = ['' for _ in range(9)]
    current_player = 'O'
    player_1_moves = set()
    player_2_moves = set()
    moves_sequence = []

def review_game(game_data):
    global board, current_player, player_1_moves, player_2_moves, moves_sequence
    player_1 = game_data['player_1']
    player_2 = game_data['player_2']
    winner = game_data['winner']
    for i in range(1, 10):
        if game_data[f'move_{i}'] == None:
            break
        moves_sequence.append(game_data[f'move_{i}'])
    return 
    

In [4]:
@app.route('/welcome', methods=['GET', 'POST'])
def enter_player_name():
    global player_1, player_2
    if request.method == 'POST':
        if 'player_1' in request.form and 'player_2' in request.form:
            player_1 = request.form['player_1']
            player_2 = request.form['player_2']
            print(player_1, player_2)
            print('Game started')
            return redirect(url_for('play_game'))
    return render_template('welcome.html')


@app.route('/game', methods=['GET', 'POST'])
def play_game():
    global board, player_1, player_2, current_player, player_1_moves, player_2_moves, moves_sequence

    state = get_game_state()
    if state == 3:
        db.history.insert_one({
            'match_timestamp': datetime.datetime.now(),
            'player_1': player_1,
            'player_2': player_2,
            'is_tie': True,
            'winner': None,
            'loser': None,
            'player_1_moves': list(player_1_moves),
            'player_2_moves': list(player_2_moves),
            'total_moves': len(moves_sequence),
            **{f'move_{i+1}': moves_sequence[i] if i < len(moves_sequence) else None for i in range(9)}
        })
        reset_game()
        return redirect(url_for('play_game'))
    elif state:
        db.history.insert_one({
            'match_timestamp': datetime.datetime.now(),
            'player_1': player_1,
            'player_2': player_2,
            'is_tie': False,
            'winner': player_1 if state == 1 else player_2, #! Check this tie condition
            'loser': player_2 if state == 1  else player_1,
            'player_1_moves': list(player_1_moves),
            'player_2_moves': list(player_2_moves),
            'total_moves': len(moves_sequence),
            **{f'move_{i+1}': moves_sequence[i] if i < len(moves_sequence) else None for i in range(9)}
        })
        reset_game()
        return redirect(url_for('play_game'))

    if request.method == 'POST':
        index = int(request.form['index'])
        if board[index] == '':
            moves_sequence.append(index)
            if current_player == 'O':
                player_1_moves.add(index)
            else:
                player_2_moves.add(index)

            board[index] = current_player
            state = get_game_state()
            current_player = 'X' if current_player == 'O' else 'O'
            return render_template('board.html', board=board, player_1=player_1, player_2=player_2, current_player=current_player, state=state)

    return render_template('board.html', board=board, player_1=player_1, player_2=player_2, current_player=current_player, state=state)

@app.route('/history')
def show_history():
    history = db.history.find()
    return render_template('history.html', history=history)

@app.route('/handle_action', methods=['POST'])
def handle_action():
    global board, current_player, player_1_moves, player_2_moves, moves_sequence
    data = request.get_json()
    game_data = db.history.find_one({"_id": ObjectId(data.get('id'))})
    player_1 = game_data['player_1']
    player_2 = game_data['player_2']
    winner = game_data['winner']
    for i in range(1, 10):
        if game_data[f'move_{i}'] == None:
            break
        moves_sequence.append(game_data[f'move_{i}'])
    return render_template('review.html', board=[], player_1=player_1, player_2=player_2)

@app.route('/')
def index():
    return redirect(url_for('enter_player_name'))

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Oct/2024 22:31:30] "GET /history HTTP/1.1" 200 -
